## Importing Libraries and models

In [1]:
%%capture
!pip install wandb

In [2]:
!nvidia-smi

Sat Oct 18 06:00:06 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   59C    P0             105W / 300W |  15603MiB / 81920MiB |     99%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import wandb
import random
import pandas as pd
import torch
import time
import numpy as np
import torch.nn as nn
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

[1760767210.903159] [bfd74565809f:5601 :f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print(device)

cuda


## Load Dataset

In [6]:
class Language:
    def __init__(self, name):
        self.name = name
        self.char2index = {'#': 0, '$': 1, '^': 2}   # '^': start of sequence, '$' : unknown char, '#' : padding
        self.index2char = {0: '#', 1: '$', 2: '^'}
        self.vocab_size = 3  # Count

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.vocab_size
            self.index2char[self.vocab_size] = char
            self.vocab_size += 1

    def encode(self, s):
        return [self.char2index[ch] for ch in s]

    def decode(self, l):
        return ''.join([self.index2char[i] for i in l])

    def vocab(self):
        return self.char2index.keys()


In [7]:
# returns maximum length of input and output words
def maxLength(data):
    ip_mlen, op_mlen = 0, 0

    for i in range(len(data)):
        input = data[0][i]
        output = data[1][i]
        if(len(input)>ip_mlen):
            ip_mlen=len(input)

        if(len(output)>op_mlen):
            op_mlen=len(output)

    return ip_mlen, op_mlen

In [8]:
import pandas as pd

def getMaxLengthValues(lang):
    base_path = f"../../aks_dataset/{lang}"
    
    # Load datasets
    train_df = pd.read_csv(f"{base_path}/train.csv", header=None)
    val_df = pd.read_csv(f"{base_path}/valid.csv", header=None)
    test_df = pd.read_csv(f"{base_path}/test.csv", header=None)

    # Initialize language vocabularies
    input_lang = Language('eng')
    output_lang = Language(lang)
    
    # Build vocabulary only from train data
    for _, row in train_df.iterrows():
        input_lang.addWord(str(row[0]))
        output_lang.addWord(str(row[1]))
    
    # Compute max input/output lengths for each split
    m1, m01 = maxLength(train_df)
    m2, m02 = maxLength(test_df)
    m3, m03 = maxLength(val_df)

    # Return the largest values across all splits
    return max(m1, m2, m3), max(m01, m02, m03)

# Example usage
input_max_len, output_max_len = getMaxLengthValues('hin')
print(input_max_len, output_max_len)

29 26


In [9]:
input_shape = 0
def preprocess(data, input_lang, output_lang, input_max_len, output_max_len, s=''):

    unknown = input_lang.char2index['$']

    n = len(data)
    input = torch.zeros((n, input_max_len + 1), device = device)
    output = torch.zeros((n, output_max_len + 2), device = device)

    for i in range(n):

        inp = data[0][i].ljust(input_max_len + 1, '#')
        op = '^' + data[1][i]       # add start symbol to output
        op = op.ljust(output_max_len + 2, '#')

        for index, char in enumerate(inp):
            if char in input_lang.char2index:
                input[i][index] = input_lang.char2index[char]
            else:
                input[i][index] = unknown

        for index, char in enumerate(op):
            if char in output_lang.char2index:
                output[i][index] = output_lang.char2index[char]
            else:
                output[i][index] = unknown

    print(s, ' dataset')
    print(input.shape)
    print(output.shape)

    return TensorDataset(input.to(torch.int32), output.to(torch.int32))

In [10]:
def load_prepare_data(lang):
    train_df = pd.read_csv(f"../../aks_dataset/{lang}/train.csv", header = None)
    val_df = pd.read_csv(f"../../aks_dataset/{lang}/valid.csv", header = None)
    test_df = pd.read_csv(f"../../aks_dataset/{lang}/test.csv", header = None)

    input_lang = Language('eng')
    output_lang = Language(lang)

    # create vocablury
    for i in range(len(train_df)):
        input_lang.addWord(train_df[0][i]) # 'eng'
        output_lang.addWord(train_df[1][i]) # 'hin'

    # encode the datasets
    train_data = preprocess(train_df, input_lang, output_lang,input_max_len, output_max_len, 'train')
    val_data = preprocess(val_df, input_lang, output_lang,input_max_len, output_max_len, 'validation')
    test_data = preprocess(test_df, input_lang, output_lang,input_max_len, output_max_len, 'test')

    return train_data, val_data, test_data, input_lang, output_lang


train_data, val_data, test_data, input_lang, output_lang = load_prepare_data('hin')

train  dataset
torch.Size([100000, 30])
torch.Size([100000, 28])
validation  dataset
torch.Size([6357, 30])
torch.Size([6357, 28])
test  dataset
torch.Size([10112, 30])
torch.Size([10112, 28])


In [11]:
print(input_lang.decode(train_data[23][0].tolist()))
output_lang.decode(train_data[23][1].tolist())

urjapurna#####################


'^उर्जापूर्ण#################'

In [12]:
train_data[23][1]

tensor([ 2, 42, 15, 18,  8, 12,  3, 29, 15, 18, 43,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0], device='cuda:0',
       dtype=torch.int32)

In [13]:
with open('api_key.txt') as input_file:
    key = input_file.read()
key = key.split('\n')[0].strip()

In [14]:
wandb.login(key =key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sshejole132 (sshejole132-iit-bombay) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# seq2seq tranformer model

### hyperparameter settings

In [15]:
n_embd = 64
batch_size = 256
learning_rate = 1e-3
n_head = 4 # other options factors of 32 like 2, 8
n_layers = 2
dropout = 0.2
epochs = 50

# encoder specific detail
input_vocab_size = input_lang.vocab_size
encoder_block_size = len(train_data[0][0])

# decoder specific detail
output_vocab_size = output_lang.vocab_size
decoder_block_size = len(train_data[0][1])

### Encoder model

In [16]:
class Head(nn.Module):
    """ one self-attention head """

    def __init__(self, n_embd, d_k, dropout, mask=0): # d_k is dimention of key , nomaly d_k = n_embd / 4
        super().__init__()
        self.mask = mask
        self.key = nn.Linear(n_embd, d_k, bias=False, device=device)
        self.query = nn.Linear(n_embd, d_k, bias=False, device=device)
        self.value = nn.Linear(n_embd, d_k, bias=False, device=device)
        if mask:
            self.register_buffer('tril', torch.tril(torch.ones(encoder_block_size, encoder_block_size, device=device)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output = None):
        B,T,C = x.shape

        if encoder_output is not None:
            k = self.key(encoder_output)
            Be, Te, Ce = encoder_output.shape
        else:
            k = self.key(x) # (B,T,d_k)

        q = self.query(x) # (B,T,d_k)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,T)

        if self.mask:
            if encoder_output is not None:
                wei = wei.masked_fill(self.tril[:T, :Te] == 0, float('-inf')) # (B,T,T)
            else:
                wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # perform weighted aggregation of values
        if encoder_output is not None:
            v = self.value(encoder_output)
        else:
            v = self.value(x)
        out = wei @ v # (B,T,C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple self attention heads in parallel """

    def __init__(self, n_embd, num_head, d_k, dropout, mask=0):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, d_k, dropout, mask) for _ in range(num_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output=None):
        out = torch.cat([h(x, encoder_output) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ multiple self attention heads in parallel """

    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class encoderBlock(nn.Module):
    """ Tranformer encoder block : communication followed by computation """

    def __init__(self, n_embd, n_head, dropout):
        super().__init__()
        d_k = n_embd // n_head
        self.sa = MultiHeadAttention(n_embd, n_head, d_k, dropout)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, encoder_output=None):
        x = x + self.sa(self.ln1(x), encoder_output)
        x = x + self.ffwd(self.ln2(x))
        return x

class Encoder(nn.Module):

    def __init__(self, n_embd, n_head, n_layers, dropout):
        super().__init__()

        self.token_embedding_table = nn.Embedding(input_vocab_size, n_embd) # n_embd: input embedding dimension
        self.position_embedding_table = nn.Embedding(encoder_block_size, n_embd)
        self.blocks = nn.Sequential(*[encoderBlock(n_embd, n_head, dropout) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,n_embd)
        x = tok_emb + pos_emb # (B,T,n_embd)
        x = self.blocks(x) # apply one attention layer (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        return x


### Decoder model

In [17]:
class decoderBlock(nn.Module):
    """ Tranformer decoder block : self communication then cross communication followed by computation """

    def __init__(self, n_embd, n_head, dropout):
        super().__init__()
        d_k = n_embd // n_head
        self.sa = MultiHeadAttention(n_embd, n_head, d_k, dropout, mask = 1)
        self.ca = MultiHeadAttention(n_embd, n_head, d_k, dropout, mask = 1)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd, device=device)
        self.ln2 = nn.LayerNorm(n_embd, device=device)
        self.ln3 = nn.LayerNorm(n_embd, device=device)

    def forward(self, x_encoder_output):
        x = x_encoder_output[0]
        encoder_output = x_encoder_output[1]
        x = x + self.sa(self.ln1(x))
        x = x + self.ca(self.ln2(x), encoder_output)
        x = x + self.ffwd(self.ln3(x))
        return (x,encoder_output)

class Decoder(nn.Module):

    def __init__(self, n_embd, n_head, n_layers, dropout):
        super().__init__()

        self.token_embedding_table = nn.Embedding(output_vocab_size, n_embd) # n_embd: input embedding dimension
        self.position_embedding_table = nn.Embedding(decoder_block_size, n_embd)
        self.blocks = nn.Sequential(*[decoderBlock(n_embd, n_head=n_head, dropout=dropout) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, output_vocab_size)

    def forward(self, idx, encoder_output, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,n_embd)
        x = tok_emb + pos_emb # (B,T,n_embd)

        x =self.blocks((x, encoder_output))
        x = self.ln_f(x[0]) # (B,T,C)
        logits = self.lm_head(x) # (B,T,output_vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            temp_logits = logits.view(B*T, C)
            targets = targets.reshape(B*T)

            loss = F.cross_entropy(temp_logits, targets.long())

        # print(logits)
        # out = torch.argmax(logits)

        return logits, loss



# Training Time

## sweep config

In [18]:
# Define sweep config
sweep_configuration = {
    "method": "bayes",
    "name": "sweep",
    "metric": {"goal": "maximize", "name": "val_acc"},
    "parameters": {
        "batch_size": {"values": [64, 128, 256]},
        "epochs": {"values": [20, 40, 50, 100]},
        "lr": {"max": 0.1, "min": 0.0001},
        "n_embd": {"values": [16, 32, 64]},
        "n_head": {"values": [2, 4, 8]},
        "n_layers": {"values": [2]},
        "dropout": {"values": [0, .1, .2, .3]}
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="Tranliteration-Tranformers")

Create sweep with ID: olbj21xb
Sweep URL: https://wandb.ai/sshejole132-iit-bombay/Tranliteration-Tranformers/sweeps/olbj21xb


In [19]:
# wandb.sweep_cancel(sweep_id)
# wandb.finish()
# wandb.run.cancel()

## train function

In [20]:
from tqdm import tqdm
def train():
    run = wandb.init()

    n_embd = wandb.config.n_embd
    n_head = wandb.config.n_head
    n_layers = wandb.config.n_layers
    dropout = wandb.config.dropout
    epochs = wandb.config.epochs
    batch_size = wandb.config.batch_size
    learning_rate = wandb.config.lr


    encoder = Encoder(n_embd, n_head, n_layers, dropout)
    decoder = Decoder(n_embd, n_head, n_layers, dropout)
    encoder.to(device)
    decoder.to(device)

    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []

    # print the number of parameters in the model
    print(sum([p.numel() for p in encoder.parameters()] + [p.numel() for p in decoder.parameters()])/1e3, 'K model parameters')

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

    # create a PyTorch optimizer
    encoder_optimizer = torch.optim.AdamW(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.AdamW(decoder.parameters(), lr=learning_rate)

# print('Step | Training Loss | Validation Loss   |   Training Accuracy %  |  Validation Accuracy %')

    least_error = float('inf')
    patience = 20  # The number of epochs without improvement to wait before stopping
    no_improvement = 0

    for i in tqdm(range(epochs)):
        running_loss = 0.0
        train_correct = 0

        encoder.train()
        decoder.train()

        for j,(train_x,train_y) in enumerate(train_loader):
            train_x = train_x.to(device)
            train_y = train_y.to(device)

            encoder_optimizer.zero_grad(set_to_none=True)
            decoder_optimizer.zero_grad(set_to_none=True)

            encoder_output = encoder(train_x)
            logits, loss = decoder(train_y[:, :-1], encoder_output, train_y[:, 1:])

            encoder_optimizer.zero_grad(set_to_none=True)
            decoder_optimizer.zero_grad(set_to_none=True)
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()

            running_loss += loss
            pred_decoder_output = torch.argmax(logits, dim=-1)
            # print(pred_decoder_output, " target: ", train_y[:, 1:])
            train_correct += (pred_decoder_output == train_y[:, 1:]).sum().item()


        ## validation code
        running_loss_val, val_correct = 0, 0
        encoder.eval()
        decoder.eval()
        for j,(val_x,val_y) in enumerate(val_loader):
            val_x = val_x.to(device)
            val_y = val_y.to(device)

            encoder_output = encoder(val_x)
            logits, loss = decoder(val_y[:, :-1], encoder_output, val_y[:, 1:])

            running_loss_val += loss
            pred_decoder_output = torch.argmax(logits, dim=-1)
            val_correct += torch.sum(pred_decoder_output == val_y[:, 1:])


        if running_loss_val < least_error:
            least_error = running_loss_val
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print(f"Early stopping at epoch {i}")
            break

        wandb.log(
            {
                "train_loss": running_loss / len(train_data),
                "val_loss": (running_loss_val/len(val_data)),
                "train_acc": ((train_correct*100) / (len(train_data)* (decoder_block_size-1))),
                "val_acc": ((val_correct*100)/(len(val_data)* (decoder_block_size-1))),
            }
        )

## run sweep

In [ ]:
wandb.agent(sweep_id=sweep_id, function=train)

wandb: Agent Starting Run: 3o6074mu with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	epochs: 50
wandb: 	lr: 0.06495675751598949
wandb: 	n_embd: 64
wandb: 	n_head: 4
wandb: 	n_layers: 2


246.787 K model parameters


 72%|███████████████████████████████████████████████████████████▊                       | 36/50 [24:27<09:30, 40.78s/it]

Early stopping at epoch 36


train_acc,▁▂▃▄▄▅▅▄▄▄▄▄▅▆▂▄▇▇▆▅▄▅▃▃▃▅▃▃▃▃▄▄▅▅▇█
train_loss,█▅▅▄▄▃▃▄▃▅▄▄▄▂█▅▂▁▃▃▃▃▅▄▄▃▆▅▅▅▄▄▃▃▂▁
val_acc,▂▂▃▄▅▄▂▄▃▄▄▄▅▆▃▄█▅▄▂▃▁▁▃▃▅▂▃▃▂▃▃▆▆▆▆
val_loss,▇▆▅▅▄▅▆▆▅▅▅▄▄▃█▄▁▄▅▅▅▇▇▅▅▄█▆▆▆▅▆▃▃▄▂
train_acc,75.60189
train_loss,0.00719
val_acc,79.19354
val_loss,0.00601


...

100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [19:19<00:00, 28.99s/it]


train_acc,▁▃▃▃▃▃▃▃▄▄▄▄▅▄▄▅▅▅▅▅▅▆▆▇▇▇▇█████▇▅▄▄▅▅▅▅
train_loss,█▆▆▆▅▆▅▆▅▅▅▅▄▅▄▄▄▄▄▄▃▃▂▂▂▂▁▁▁▁▁▁▂▃▄▄▄▄▄▄
val_acc,▁▂▂▂▂▂▂▂▃▃▃▄▄▃▄▄▄▄▄▅▅▆▆▇▇▇█▇███▇▇▄▄▄▄▄▄▄
val_loss,█▇▆▆▆▇▆▆▆▆▅▅▅▅▅▅▅▄▅▄▃▃▂▂▂▂▁▁▁▁▁▂▂▅▅▅▄▄▄▅
train_acc,83.87774
train_loss,0.00216
val_acc,86.87244
val_loss,0.00176


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ay9in1mq with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 50
wandb: 	lr: 0.0006086129056137431
wandb: 	n_embd: 64
wandb: 	n_head: 8
wandb: 	n_layers: 2


246.787 K model parameters


 66%|██████████████████████████████████████████████████████▊                            | 33/50 [14:26<07:13, 25.50s/it]

# Test Time
Since this is the best model(validation accuracy) , we will train it on both train and validation data.
We will then test the model on test data

## Best Hyperparameter from validation

In [18]:
n_embd = 64
batch_size = 256
learning_rate = 0.003699
n_head = 4
n_layers = 2
dropout = 0
epochs = 20

encoder = Encoder(n_embd, n_head, n_layers, dropout)
decoder = Decoder(n_embd, n_head, n_layers, dropout)
encoder.to(device)
decoder.to(device)
print("✅ Hyperparameters set successfully")

✅ Hyperparameters set successfully


## Train on train_data + val_data

In [23]:

# print the number of parameters in the model
print(sum([p.numel() for p in encoder.parameters()] + [p.numel() for p in decoder.parameters()])/1e3, 'K model parameters')

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

# create a PyTorch optimizer
encoder_optimizer = torch.optim.AdamW(encoder.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.AdamW(decoder.parameters(), lr=learning_rate)

# print('Step | Training Loss | Validation Loss   |   Training Accuracy %  |  Validation Accuracy %')

least_error = float('inf')
patience = 20  # The number of epochs without improvement to wait before stopping
no_improvement = 0

for i in range(epochs):
    running_loss = 0.0
    train_correct = 0

    encoder.train()
    decoder.train()

    for j,(train_x,train_y) in enumerate(train_loader):
        train_x = train_x.to(device)
        train_y = train_y.to(device)

        encoder_optimizer.zero_grad(set_to_none=True)
        decoder_optimizer.zero_grad(set_to_none=True)

        encoder_output = encoder(train_x)
        logits, loss = decoder(train_y[:, :-1], encoder_output, train_y[:, 1:])

        encoder_optimizer.zero_grad(set_to_none=True)
        decoder_optimizer.zero_grad(set_to_none=True)
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        running_loss += loss
        pred_decoder_output = torch.argmax(logits, dim=-1)
        # print(pred_decoder_output, " target: ", train_y[:, 1:])
        train_correct += (pred_decoder_output == train_y[:, 1:]).sum().item()

    for j,(train_x,train_y) in enumerate(val_loader):
        train_x = train_x.to(device)
        train_y = train_y.to(device)

        encoder_optimizer.zero_grad(set_to_none=True)
        decoder_optimizer.zero_grad(set_to_none=True)

        encoder_output = encoder(train_x)
        logits, loss = decoder(train_y[:, :-1], encoder_output, train_y[:, 1:])

        encoder_optimizer.zero_grad(set_to_none=True)
        decoder_optimizer.zero_grad(set_to_none=True)
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        running_loss += loss
        pred_decoder_output = torch.argmax(logits, dim=-1)
        # print(pred_decoder_output, " target: ", train_y[:, 1:])
        train_correct += (pred_decoder_output == train_y[:, 1:]).sum().item()


    metrics = {
            "train_loss": running_loss.cpu().detach().numpy() / (len(train_data)+len(val_data)),
            "train_acc": ((train_correct*100) / ((len(train_data)+len(val_data))* (decoder_block_size-1))),
        }
    if i % 5 == 0:
        print("Step: ",i)
        print("train_loss: ", metrics["train_loss"])
        print("train_acc: ", metrics["train_acc"])

246.787 K model parameters
Step:  0
train_loss:  0.0017554496738523468
train_acc:  87.72230771346956
Step:  5
train_loss:  0.00042737022913588274
train_acc:  96.22755506524322
Step:  10
train_loss:  0.0003673466991669639
train_acc:  96.74785026948025
Step:  15
train_loss:  0.0003365819262775884
train_acc:  96.99969251009615


In [28]:
PATH = 'models/transformer-encoder.pth'
torch.save(encoder, PATH)
PATH = 'models/transformer-decoder.pth'
torch.save(decoder, PATH)

## generate output sequence

In [21]:
def generate(input):
    B, T = input.shape
    encoder_output = encoder(input)
    idx = torch.full((B, 1), 2, dtype=torch.long, device=device) # (B,1)

    # idx is (B, T) array of indices in the current context
    for _ in range(decoder_block_size-1):
        # get the predictions
        logits, loss = decoder(idx, encoder_output) # logits (B, T, vocab_size)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        idx_next = torch.argmax(logits, dim=-1, keepdim=True) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

## Check Test Accuracy

In [25]:
import torch
from torch.utils.data import DataLoader

def check(encoder, decoder, test_data, device='cpu', batch_size=64, pad_idx=0):
    """
    Compute:
      - word-level accuracy: fraction of sequences where all non-pad tokens match exactly
      - char-level accuracy: fraction of non-pad tokens that are predicted correctly
      - avg validation loss (per-batch loss averaged over batches)
    Parameters:
      encoder, decoder : your models
      test_data         : dataset (not a dataloader) or anything accepted by DataLoader
      device            : 'cpu' or 'cuda'
      batch_size        : dataloader batch size
      pad_idx           : integer index used for padding tokens; set to None to not mask pads
    """
    encoder.eval()
    decoder.eval()

    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    total_sequences = 0           # total number of sequences (examples)
    total_nonpad_tokens = 0       # total number of non-pad tokens counted across all sequences
    char_correct = 0              # count of correctly predicted tokens (char-level)
    word_exact_correct = 0        # count of sequences with exact match on all non-pad tokens
    running_loss_val = 0.0
    n_batches = 0

    with torch.no_grad():
        for val_x, val_y in test_loader:
            val_x = val_x.to(device)
            val_y = val_y.to(device)

            # generate should produce token predictions of same shape as val_y
            output = generate(val_x)  # expected shape: (batch, seq_len)
            # get logits and loss from decoder as before; decoder returns (logits, loss)
            encoder_output = encoder(val_x)
            logits, loss = decoder(val_y[:, :-1], encoder_output, val_y[:, 1:])
            # accumulate scalar loss
            running_loss_val += loss.item()
            n_batches += 1

            # Align lengths: compare output[:,1:] with val_y[:,1:]
            pred = output[:, 1:]
            target = val_y[:, 1:]

            # ensure same shape (if generate produced different length, adjust or slice)
            min_len = min(pred.size(1), target.size(1))
            pred = pred[:, :min_len]
            target = target[:, :min_len]

            if pad_idx is not None:
                mask = (target != pad_idx)          # True where token is not padding
                nonpad_count = mask.sum().item()
                # char-level correct: count positions where pred == target and target != pad
                char_correct += ( (pred == target) & mask ).sum().item()
                total_nonpad_tokens += nonpad_count

                # word-level exact: for each sequence, check equality only on non-pad positions
                # If a sequence has zero nonpad tokens (weird), treat as not correct.
                seq_equal = torch.all( ((pred == target) | (~mask)), dim=1 )
                # but we should exclude sequences with zero nonpad tokens from denominator:
                nonpad_per_seq = mask.sum(dim=1)
                valid_seq_mask = (nonpad_per_seq > 0)
                if valid_seq_mask.any():
                    word_exact_correct += seq_equal[valid_seq_mask].sum().item()
                    total_sequences += valid_seq_mask.sum().item()
                # sequences with zero nonpad tokens are ignored for word-level stats
            else:
                # no padding: count all positions
                total_nonpad_tokens += target.numel()
                char_correct += (pred == target).sum().item()

                seq_equal = torch.all(pred == target, dim=1)
                word_exact_correct += seq_equal.sum().item()
                total_sequences += pred.size(0)

    # final metrics
    avg_loss = running_loss_val / n_batches if n_batches > 0 else float('nan')
    char_acc = (char_correct / total_nonpad_tokens * 100.0) if total_nonpad_tokens > 0 else float('nan')
    word_acc = (word_exact_correct / total_sequences * 100.0) if total_sequences > 0 else float('nan')

    print(f"Validation avg loss (per-batch): {avg_loss:.6f}")
    print(f"Char-level accuracy (non-pad tokens): {char_acc:.4f}%  ({char_correct}/{total_nonpad_tokens})")
    print(f"Word-level (sequence-exact) accuracy: {word_acc:.4f}%  ({word_exact_correct}/{total_sequences})")

    return {
        "avg_loss": avg_loss,
        "char_acc": char_acc,
        "word_acc": word_acc,
        "char_correct": char_correct,
        "total_nonpad_tokens": total_nonpad_tokens,
        "word_exact_correct": word_exact_correct,
        "total_sequences": total_sequences
    }


In [31]:
def check2(encoder, decoder, test_data, device='cpu', batch_size=64, pad_idx=0, max_bleu_n=4):
    """
    Compute:
      - word-level accuracy: fraction of sequences where all non-pad tokens match exactly
      - char-level accuracy: fraction of non-pad tokens that are predicted correctly
      - CER (Character Error Rate)
      - BLEU (sentence-level, character-based)
      - avg validation loss (per-batch loss averaged over batches)
    """
    encoder.eval()
    decoder.eval()

    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    total_sequences = 0
    total_nonpad_tokens = 0
    char_correct = 0
    word_exact_correct = 0
    running_loss_val = 0.0
    n_batches = 0

    total_edit_distance = 0
    total_chars_for_cer = 0
    total_bleu = 0.0

    with torch.no_grad():
        for val_x, val_y in test_loader:
            val_x = val_x.to(device)
            val_y = val_y.to(device)

            # generate predicted sequences
            output = generate(val_x)  # shape: (batch, seq_len)

            # compute decoder loss as before
            encoder_output = encoder(val_x)
            logits, loss = decoder(val_y[:, :-1], encoder_output, val_y[:, 1:])
            running_loss_val += loss.item()
            n_batches += 1

            pred = output[:, 1:]
            target = val_y[:, 1:]

            # align lengths
            min_len = min(pred.size(1), target.size(1))
            pred = pred[:, :min_len]
            target = target[:, :min_len]

            # convert tensors to lists of integers
            pred_list = pred.tolist()
            target_list = target.tolist()

            for p_seq, t_seq in zip(pred_list, target_list):
                # mask padding if needed
                if pad_idx is not None:
                    mask = [t != pad_idx for t in t_seq]
                    p_seq_masked = [p for p, m in zip(p_seq, mask) if m]
                    t_seq_masked = [t for t, m in zip(t_seq, mask) if m]
                else:
                    p_seq_masked = p_seq
                    t_seq_masked = t_seq

                if len(t_seq_masked) == 0:
                    continue

                # char-level correct
                char_correct += sum(p == t for p, t in zip(p_seq_masked, t_seq_masked))
                total_nonpad_tokens += len(t_seq_masked)

                # word-level exact
                if p_seq_masked == t_seq_masked:
                    word_exact_correct += 1
                total_sequences += 1

                # CER
                total_edit_distance += levenshtein_distance(p_seq_masked, t_seq_masked)
                total_chars_for_cer += len(t_seq_masked)

                # BLEU
                total_bleu += sentence_bleu(t_seq_masked, p_seq_masked, max_n=max_bleu_n)

    avg_loss = running_loss_val / n_batches if n_batches > 0 else float('nan')
    char_acc = char_correct / total_nonpad_tokens * 100.0 if total_nonpad_tokens > 0 else float('nan')
    word_acc = word_exact_correct / total_sequences * 100.0 if total_sequences > 0 else float('nan')
    cer = total_edit_distance / total_chars_for_cer if total_chars_for_cer > 0 else float('nan')
    avg_bleu = total_bleu / total_sequences if total_sequences > 0 else float('nan')

    print(f"Validation avg loss (per-batch): {avg_loss:.6f}")
    print(f"Char-level accuracy (non-pad tokens): {char_acc:.4f}%  ({char_correct}/{total_nonpad_tokens})")
    print(f"Word-level (sequence-exact) accuracy: {word_acc:.4f}%  ({word_exact_correct}/{total_sequences})")
    print(f"Character Error Rate (CER): {cer:.4f}")
    print(f"Average BLEU (char-based): {avg_bleu:.4f}")

    return {
        "avg_loss": avg_loss,
        "char_acc": char_acc,
        "word_acc": word_acc,
        "cer": cer,
        "bleu": avg_bleu,
        "char_correct": char_correct,
        "total_nonpad_tokens": total_nonpad_tokens,
        "word_exact_correct": word_exact_correct,
        "total_sequences": total_sequences
    }


In [26]:
import torch

# Make sure Encoder and Decoder class definitions are already imported or defined above this line!

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load saved encoder and decoder
encoder = torch.load('models/transformer-encoder.pth', map_location=device)
decoder = torch.load('models/transformer-decoder.pth', map_location=device)

# Put them in eval mode before testing
encoder.eval()
decoder.eval()

print("Models loaded successfully on", device)

Models loaded successfully on cuda


In [27]:
import pickle
import os

# Make sure the directory exists
os.makedirs("models", exist_ok=True)

# Save input_lang
with open(os.path.join("models", "input_lang.pkl"), "wb") as f:
    pickle.dump(input_lang, f)

# Save output_lang
with open(os.path.join("models", "output_lang.pkl"), "wb") as f:
    pickle.dump(output_lang, f)

print("input_lang and output_lang saved successfully in 'models/' directory.")

input_lang and output_lang saved successfully in 'models/' directory.


In [28]:
print(len(train_data[0][0]))
print(len(train_data[0][1]))
print(len(val_data[0][0]))
print(len(val_data[0][1]))
print(len(test_data[0][0]))
print(len(test_data[0][1]))

30
28
30
28
30
28


In [29]:
print(input_vocab_size)
print(output_vocab_size)

29
67


In [30]:
# encoder specific detail
input_vocab_size = input_lang.vocab_size
encoder_block_size = len(train_data[0][0])

# decoder specific detail
output_vocab_size = output_lang.vocab_size
decoder_block_size = len(train_data[0][1])

check(encoder, decoder, test_data, device)

Validation avg loss (per-batch): 0.112573
Char-level accuracy (non-pad tokens): 71.6944%  (49819/69488)
Word-level (sequence-exact) accuracy: 42.0392%  (4251/10112)


{'avg_loss': 0.1125725770156972,
 'char_acc': 71.69439327653696,
 'word_acc': 42.03916139240506,
 'char_correct': 49819,
 'total_nonpad_tokens': 69488,
 'word_exact_correct': 4251,
 'total_sequences': 10112}

In [32]:
check2(encoder, decoder, test_data, device)

Validation avg loss (per-batch): 0.112573
Char-level accuracy (non-pad tokens): 71.6944%  (49819/69488)
Word-level (sequence-exact) accuracy: 42.0392%  (4251/10112)
Character Error Rate (CER): 0.2009
Average BLEU (char-based): 73.0857


{'avg_loss': 0.1125725770156972,
 'char_acc': 71.69439327653696,
 'word_acc': 42.03916139240506,
 'cer': 0.20089799677642184,
 'bleu': 73.08571634442133,
 'char_correct': 49819,
 'total_nonpad_tokens': 69488,
 'word_exact_correct': 4251,
 'total_sequences': 10112}

In [18]:
import torch
from torch.utils.data import DataLoader

def transliterate_word(word, input_lang, output_lang,
                       encoder, decoder, device='cpu',
                       input_max_len=30, output_max_len=30):
    """
    Transliterates a single English word using your trained encoder-decoder setup.

    Args:
        word (str): English word to transliterate
        input_lang, output_lang: objects with char2index (and optionally index2char)
        encoder, decoder: trained models
        device (str): 'cpu' or 'cuda'
        input_max_len, output_max_len: max sequence lengths (used for padding)

    Returns:
        predicted_text (str): transliterated output
    """
    encoder.eval()
    decoder.eval()

    # build inverse map for output indices
    if hasattr(output_lang, 'index2char'):
        idx2char = output_lang.index2char
    else:
        idx2char = {v: k for k, v in output_lang.char2index.items()}

    # unknown and padding chars
    unknown = input_lang.char2index.get('$', 0)
    pad_char = '#'

    # pad the word to match training preprocessing
    inp = word.ljust(input_max_len + 1, pad_char)
    input_tensor = torch.zeros((1, input_max_len + 1), dtype=torch.long, device=device)
    for i, ch in enumerate(inp):
        input_tensor[0, i] = input_lang.char2index.get(ch, unknown)

    # generate prediction
    with torch.no_grad():
        output_idx = generate(input_tensor)  # (1, seq_len)
        output_idx = output_idx[0].cpu().tolist()

    # convert to text: skip start (^), pad (#), unknown ($)
    predicted_chars = []
    for i in output_idx:
        ch = idx2char.get(i, '')
        if ch in ['#', '^', '$']:
            continue
        predicted_chars.append(ch)

    predicted_text = ''.join(predicted_chars).strip()

    print(f"Input : {word}")
    print(f"Output: {predicted_text}")
    return predicted_text

In [19]:
transliterate_word("ram", input_lang, output_lang, encoder, decoder, device)

/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [15,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [15,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [15,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [15,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [15,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [15,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDim

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [38]:
import torch
from torch.utils.data import DataLoader

def build_index2char(lang):
    """Return index->char dict for a language object that has char2index.
       If lang already has index2char attribute, use it."""
    if hasattr(lang, 'index2char'):
        return lang.index2char
    inv = {idx: ch for ch, idx in lang.char2index.items()}
    return inv

def test_word(word,
              input_lang, output_lang,
              encoder, decoder, generate,
              input_max_len, output_max_len,
              device='cpu'):
    """
    Test a single English word using your preprocess function.
    - word: string (e.g., 'kumar')
    - input_lang, output_lang: objects with char2index (and optionally index2char)
    - encoder, decoder, generate: trained model pieces (generate takes batched inputs)
    - input_max_len, output_max_len: ints used in preprocess
    - device: 'cpu' or 'cuda'
    Returns predicted string (transliteration).
    """
    encoder.eval()
    decoder.eval()

    # prepare a dummy output (preprocess needs data[1])
    data = ( [word], [''] )  # input list, output list (empty target used only for shaping)

    # create dataset (uses device internally when creating tensors)
    ds = preprocess(data, input_lang, output_lang, input_max_len, output_max_len, s=f"single '{word}'")

    # DataLoader for a single example
    loader = DataLoader(ds, batch_size=1, shuffle=False)

    # build index->char mapping for the output language
    idx2char = build_index2char(output_lang)

    # run inference
    with torch.no_grad():
        for inp_batch, _ in loader:
            inp_batch = inp_batch.to(device).long()   # shape: (1, input_len+1)
            # If your encoder expects shape (batch, seq_len) this is fine.
            # Use generate() to produce predicted token ids (batch, out_seq_len)
            pred_ids = generate(inp_batch)            # assume tensor of token ids

            # if generate returns logits, convert to ids
            if pred_ids.dim() == 3:  # (batch, seq_len, vocab)
                pred_ids = pred_ids.argmax(-1)

            pred_ids = pred_ids[0].cpu().tolist()  # list of token ids for this single example

            # Map ids to chars, filter out start '^' and padding '#'
            # In your preprocessing: '^' is start, '#' is padding, '$' is unknown
            # Keep characters until end (we will ignore '#' and '^' if present)
            chars = []
            for tid in pred_ids:
                ch = idx2char.get(tid, None)
                if ch is None:
                    # unknown index: skip or replace with '?'
                    continue
                if ch == '^':    # skip start symbol if it appears
                    continue
                if ch == '#':    # skip padding
                    continue
                # optional: stop at EOS if you have one (e.g., ch == '$' used differently)
                # append otherwise
                chars.append(ch)

            predicted = ''.join(chars).strip()

            print(f"Input : {word}")
            print(f"Output: {predicted}")

            return predicted

    # if loader was empty for some reason
    return ''

In [41]:
test_word("word", input_lang, output_lang, encoder, decoder, generate, input_max_len, output_max_len)

IndexError: list index out of range

# Plotting the Attention HeatMaps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.font_manager import FontProperties
tel_font = FontProperties(fname = 'TiroDevanagariHindi-Regular.ttf')
# Assuming you have attention_weights of shape (batch_size, output_sequence_length, batch_size, input_sequence_length)
# and prediction_matrix of shape (batch_size, output_sequence_length)
# and input_matrix of shape (batch_size, input_sequence_length)

# Define the grid dimensions
rows = int(np.ceil(np.sqrt(12)))
cols = int(np.ceil(12 / rows))

# Create a figure and subplots
fig, axes = plt.subplots(rows, cols, figsize=(9, 9))

for i, ax in enumerate(axes.flatten()):
    if i < 12:
        prediction = [opLang.index2char[j.item()] for j in pred[i+1]]

        pred_word=""
        input_word=""

        for j in range(len(prediction)):
            # Ignore padding
            if(prediction[j] != '#'):
                pred_word += prediction[j]
            else :
                break
        input_seq = [ipLang.index2char[j.item()] for j in testData[i][0]]

        for j in range(len(input_seq)):
            if(input_seq[j] != '#'):
                    input_word += input_seq[j]
            else :
                break
        attn_weights = atten_weights[i, :len(pred_word), :len(input_word)].detach().cpu().numpy()
        ax.imshow(attn_weights.T, cmap='hot', interpolation='nearest')
        ax.xaxis.set_label_position('top')
        ax.set_title(f'Example {i+1}')
        ax.set_xlabel('Output predicted')
        ax.set_ylabel('Input word')
        ax.set_xticks(np.arange(len(pred_word)))
        ax.set_xticklabels(pred_word, rotation = 90, fontproperties = tel_font,fontdict={'fontsize':8})
        ax.xaxis.tick_top()

        ax.set_yticks(np.arange(len(input_word)))
        ax.set_yticklabels(input_word, rotation=90)



# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()
wandb.init(project='CS6910_Assignment_3')

# Convert the matplotlib figure to an image
fig.canvas.draw()
image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

# Log the image in wandb
wandb.log({"attention_heatmaps": [wandb.Image(image)]})